In [ ]:
%matplotlib inline 
# %matplotlib notebook # for interactive

For high dpi displays.

In [ ]:
%config InlineBackend.figure_format = 'retina'

# 0. Note

- This notebook shows an example of how to conduct equation of state fitting for the pressure-volume-temperature data using `pytheos`.

- Advantage of using `pytheos` is that you can apply different pressure scales and different equations without much coding.  

- We use data on SiC from [Nisr et al. (2017, JGR-Planet)]().

# 1. Setup

In [ ]:
import numpy as np
import uncertainties as uct
import pandas as pd
from uncertainties import unumpy as unp
import matplotlib.pyplot as plt
import pytheos as eos

# 2. Setups for fitting with two different gold pressure scales

Equations of state of gold from Fei et al. (2007, PNAS) and Dorogokupets and Dewaele (2007, HPR).  These equations are provided from `pytheos` as built-in classes.

In [ ]:
au_eos = {'Fei2007': eos.gold.Fei2007bm3(), 'Dorogokupets2007': eos.gold.Dorogokupets2007()}

Because we use Birch-Murnaghan eos version of Fei2007 and Dorogokupets2007 used Vinet eos, we create a dictionary to provide different static compression eos for the different pressure scales used.

In [ ]:
st_model = {'Fei2007': eos.BM3Model, 'Dorogokupets2007': eos.VinetModel}

In [ ]:
k0 = {'Fei2007': 241.2, 'Dorogokupets2007': 243.0}
k0p = {'Fei2007': 2.84, 'Dorogokupets2007': 2.68}

Initial guess:

In [ ]:
gamma0 =
q = 
theta0 = 

Physical constants for different materials

In [ ]:
v0 = 
n = 
z = 

# 3. Setup data (3C)

Data file is in `csv` format.

In [ ]:
data = pd.read_csv('./data/3C-HiTEOS-final.csv')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
v_std = 
temp = unp.uarray(data['T(3C)'], data['sT(3C)'])
v = unp.uarray(data['V(3C)'], data['sV(3C)'])

`pytheos` provides `plot.thermal_data` function to show the data distribution in the P-V and P-T spaces.

In [ ]:
key = 'Dorogokupets2007' #'Fei2007' # 

Calculate pressure and plot the data

In [ ]:
p = au_eos[key].cal_p(v_std, temp)
eos.plot.thermal_data({'p': p, 'v': v, 'temp': temp}, title=key)

# 4. Data fitting with constq equation (3C)

The cell below shows fitting using constant q assumptions for the thermal part of eos.

Normally weight for each data point can be calculated from $\sigma(P)$.  In this case, using `uncertainties`, we can easily propagate the temperature and volume uncertainties to get the value.

Calculate pressure

In [ ]:
p = au_eos[key].cal_p(v_std, temp)

Add prefix to the parameters. This is important to distinguish thermal and static parameters

In [ ]:
eos_st = st_model[key](prefix='st_') 
eos_th = eos.ConstqModel(n, z, prefix='th_')

Define initial values for parameters

In [ ]:
params = eos_st.make_params(v0=v0, k0=k0[key], k0p=k0p[key])
params += eos_th.make_params(v0=v0, gamma0=gamma0, q=q, theta0=theta0)

Construct PVT eos.  Here we take advantage of lmfit to combine any formula of static and thermal eos's

Fix static parameters and some other well known parameters

Calculate weights.  Setting it None results in unweighted fitting

Conduct fitting.

Output the fitting results

In [ ]:
print('********'+key)
print(fit_result.fit_report())
# plot fitting results
eos.plot.thermal_fit_result(fit_result, p_err=unp.std_devs(p), v_err=unp.std_devs(v), title=key)

The warning message above is because the static EOS does not need temperature.  `lmfit` generates warning if an assigned independent variable is not used in fitting for any components.